In [1]:
import pandas as pd
import numpy as np

# Demographic Variables and Sample Weights (DEMO_H)
RIAGENDR - Gender
RIDAGEYR - Age in years at screening
female = 2

# Medical Conditions (MCQ_H)
SEQN - Respondent sequence number
- MCQ230A - What kind of cancer (Code 14 is Breast, many codes many different cancers)
- MCQ240E - Age when breast cancer first diagnosed
- MCQ220 - Ever told you had cancer or malignancy
no = 2


# Complete Blood Count with 5-part Differential - Whole Blood (CBC_H)
SEQN - Respondent sequence number
LBXWBCSI - White blood cell count (1000 cells/uL)
LBXLYPCT - Lymphocyte percent (%)
LBXMOPCT - Monocyte percent (%)
LBXNEPCT - Segmented neutrophils percent (%)
LBXEOPCT - Eosinophils percent (%)
LBXBAPCT - Basophils percent (%)
LBDLYMNO - Lymphocyte number (1000 cells/uL)
LBDMONO - Monocyte number (1000 cells/uL)
LBDNENO - Segmented neutrophils num (1000 cell/uL)
LBDEONO - Eosinophils number (1000 cells/uL)
LBDBANO - Basophils number (1000 cells/uL)
LBXRBCSI - Red blood cell count (million cells/uL)
LBXHGB - Hemoglobin (g/dL)
LBXHCT - Hematocrit (%)
LBXMCVSI - Mean cell volume (fL)
LBXMCHSI - Mean cell hemoglobin (pg)
LBXMC - MCHC (g/dL)
LBXRDW - Red cell distribution width (%)
LBXPLTSI - Platelet count (1000 cells/uL)
LBXMPSI - Mean platelet volume (fL)
 

# Standard Biochemistry Profile (BIOPRO_H)
SEQN - Respondent sequence number
LBDSALSI - Albumin (g/L)
LBXSAPSI - Alkaline phosphatase (IU/L)
LBXSASSI - Aspartate aminotransferase AST (IU/L)
LBXSATSI - Alanine aminotransferase ALT (IU/L)
LBXSBU - Blood urea nitrogen (mg/dL)
LBDSBUSI - Blood urea nitrogen (mmol/L)
LBXSC3SI - Bicarbonate (mmol/L)
LBDSCASI - Total calcium (mmol/L)
LBDSCHSI - Cholesterol (mmol/L)
LBXSCK - Creatine Phosphokinase(CPK) (IU/L)
LBXSCLSI - Chloride (mmol/L)
LBDSCRSI - Creatinine (umol/L)
LBDSGBSI - Globulin (g/L)
LBDSGLSI - Glucose, refrigerated serum (mmol/L)
LBXSGTSI - Gamma glutamyl transferase (U/L)
LBDSIRSI - Iron, refrigerated serum (umol/L)
LBXSKSI - Potassium (mmol/L)
LBXSLDSI - Lactate dehydrogenase (U/L)
LBXSNASI - Sodium (mmol/L)
LBXSOSSI - Osmolality (mmol/Kg)
LBDSPHSI - Phosphorus (mmol/L)
LBDSTBSI - Total bilirubin (umol/L)
LBDSTPSI - Total protein (g/L)
LBDSTRSI - Triglycerides, refrigerated (mmol/L)
LBDSUASI - Uric acid (umol/L)

# GET MORE DATA

In [2]:
for i in ['A']:    
    df_demo= pd.read_sas('./NHANES 2013-2014/other NHANES/DEMO_'+i+'.XPT')
    # selecting specific columns
    # selecting gender: women SEQN and Age
    df_demo = df_demo[(df_demo['RIAGENDR'] == 2 ) & (df_demo['RIDAGEYR'] > 10) ][['SEQN', 'RIDAGEYR']]
    df_demo = df_demo.rename(columns={'RIDAGEYR': 'Age'})

    df_med = pd.read_sas('./NHANES 2013-2014/other NHANES/MCQ_'+i+'.XPT')
    # match only dose SEQN matching the one inf df_demo (seqn variable)
    df_med = df_med.loc[df_med['SEQN'].isin(df_demo['SEQN'])]
    df_med = df_med[['SEQN', 'MCQ230A', 'MCQ240E', 'MCQ220']]
    # add column breast cancer 'Diagnosis' 1 no 0
    df_med['Breast Cancer'] = np.where(df_med['MCQ230A'] == 14,1,0)

    # subset to have women that were diagnosed with breast cancer 
    df_med = df_med[(df_med['Breast Cancer'] == 1)]
    # changing the column name to diagnosis age and saving the dataset
    df_med = df_med.rename(columns={'MCQ240E' : 'Diagnosis Age'})[['SEQN', 'Diagnosis Age', 'Breast Cancer']]
    # merge df_med with df_demo
    df = pd.merge(df_med, df_demo, on='SEQN', how='inner')

    df_blood = pd.read_sas('./NHANES 2013-2014/other NHANES/CBC_'+i+'.XPT')
    # subset to SEQN from df
    df_blood = df_blood.loc[df_blood['SEQN'].isin(df['SEQN'])]
    col_dict = {
    'LBXWBCSI' : 'BL_White blood cell count (1000 cells/uL)',
    'LBXLYPCT' : 'BL_Lymphocyte percent (%)',
    'LBXMOPCT' : 'BL_Monocyte percent (%)',
    'LBXNEPCT' : 'BL_Segmented neutrophils percent (%)',
    'LBXEOPCT' : 'BL_Eosinophils percent (%)',
    'LBXBAPCT' : 'BL_Basophils percent (%)',
    'LBDLYMNO' : 'BL_Lymphocyte number (1000 cells/uL)',
    'LBDMONO' : 'BL_Monocyte number (1000 cells/uL)',
    'LBDNENO' : 'BL_Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO' : 'BL_Eosinophils number (1000 cells/uL)',
    'LBDBANO' : 'BL_Basophils number (1000 cells/uL)',
    'LBXRBCSI' : 'BL_Red blood cell count (million cells/uL)',
    'LBXHGB' : 'BL_Hemoglobin (g/dL)',
    'LBXHCT' : 'BL_Hematocrit (%)',
    'LBXMCVSI' : 'BL_Mean cell volume (fL)',
 #   'LBXMCHSI' : 'BL_Mean cell hemoglobin (pg)',
 #   'LBXMC' : 'BL_MCHC (g/dL)',
    'LBXRDW' : 'BL_Red cell distribution width (%)',
    'LBXPLTSI' : 'BL_Platelet count (1000 cells/uL)',
    'LBXMPSI' : 'BL_Mean platelet volume (fL)'
    }
    df_blood = df_blood.rename(columns = col_dict)
    # merge df_blood to df
    df = pd.merge(df, df_blood, how='inner', on=['SEQN'])

    df_bio = pd.read_sas('./NHANES 2013-2014/other NHANES/BIOPRO_'+i+'.XPT')
    col_dict = {
    'SEQN' : 'SEQN',
    'LBDSALSI' : 'BIO_Albumin (g/L)',
    #'LBXSAPSI' : 'BIO_Alkaline phosphatase (IU/L)',
    'LBXSASSI' : 'BIO_Aspartate aminotransferase AST (IU/L)',
    'LBXSATSI' : 'BIO_Alanine aminotransferase ALT (IU/L)',
    'LBXSBU' : 'BIO_Blood urea nitrogen (mg/dL)',
    'LBDSBUSI' : 'BIO_Blood urea nitrogen (mmol/L)',
    'LBXSC3SI' : 'BIO_Bicarbonate (mmol/L)',
    'LBDSCASI' : 'BIO_Total calcium (mmol/L)',
    'LBDSCHSI' : 'BIO_Cholesterol (mmol/L)',
    #'LBXSCK' : 'BIO_Creatine Phosphokinase(CPK) (IU/L)',
    'LBXSCLSI' : 'BIO_Chloride (mmol/L)',
    'LBDSCRSI' : 'BIO_Creatinine (umol/L)',
    'LBDSGBSI' : 'BIO_Globulin (g/L)',
    'LBDSGLSI' : 'BIO_Glucose, refrigerated serum (mmol/L)',
    'LBXSGTSI' : 'BIO_Gamma glutamyl transferase (U/L)',
    'LBDSIRSI' : 'BIO_Iron, refrigerated serum (umol/L)',
    'LBXSKSI' : 'BIO_Potassium (mmol/L)',
   # 'LBXSLDSI' : 'BIO_Lactate dehydrogenase (U/L)',
    'LBXSNASI' : 'BIO_Sodium (mmol/L)',
    'LBXSOSSI' : 'BIO_Osmolality (mmol/Kg)',
    'LBDSPHSI' : 'BIO_Phosphorus (mmol/L)',
    'LBDSTBSI' : 'BIO_Total bilirubin (umol/L)',
    'LBDSTPSI' : 'BIO_Total protein (g/L)',
    'LBDSTRSI' : 'BIO_Triglycerides, refrigerated (mmol/L)',
    'LBDSUASI' : 'BIO_Uric acid (umol/L)'}
    # select the columns i want to keep
    df_bio = df_bio[list(col_dict.keys())].rename(columns=col_dict)
    # merge df_bio with df
    df = pd.merge(df, df_bio, how='inner', on=['SEQN'])
    
    df_final = df.copy()

In [3]:
for i in ['B','C','D','E','F','G','H','I']:    
    df_demo= pd.read_sas('./NHANES 2013-2014/other NHANES/DEMO_'+i+'.XPT')
    # selecting specific columns
    # selecting gender: women SEQN and Age
    df_demo = df_demo[(df_demo['RIAGENDR'] == 2 ) & (df_demo['RIDAGEYR'] > 10) ][['SEQN', 'RIDAGEYR']]
    df_demo = df_demo.rename(columns={'RIDAGEYR': 'Age'})

    df_med = pd.read_sas('./NHANES 2013-2014/other NHANES/MCQ_'+i+'.XPT')
    # match only dose SEQN matching the one inf df_demo (seqn variable)
    df_med = df_med.loc[df_med['SEQN'].isin(df_demo['SEQN'])]
    df_med = df_med[['SEQN', 'MCQ230A', 'MCQ240E', 'MCQ220']]
    # add column breast cancer 'Diagnosis' 1 no 0
    df_med['Breast Cancer'] = np.where(df_med['MCQ230A'] == 14,1,0)

    # subset to have women that were diagnosed with breast cancer 
    df_med = df_med[(df_med['Breast Cancer'] == 1)]
    # changing the column name to diagnosis age and saving the dataset
    df_med = df_med.rename(columns={'MCQ240E' : 'Diagnosis Age'})[['SEQN', 'Diagnosis Age', 'Breast Cancer']]
    # merge df_med with df_demo
    df = pd.merge(df_med, df_demo, on='SEQN', how='inner')


    df_blood = pd.read_sas('./NHANES 2013-2014/other NHANES/CBC_'+i+'.XPT')
    # subset to SEQN from df
    df_blood = df_blood.loc[df_blood['SEQN'].isin(df['SEQN'])]
    col_dict = {
    'LBXWBCSI' : 'BL_White blood cell count (1000 cells/uL)',
    'LBXLYPCT' : 'BL_Lymphocyte percent (%)',
    'LBXMOPCT' : 'BL_Monocyte percent (%)',
    'LBXNEPCT' : 'BL_Segmented neutrophils percent (%)',
    'LBXEOPCT' : 'BL_Eosinophils percent (%)',
    'LBXBAPCT' : 'BL_Basophils percent (%)',
    'LBDLYMNO' : 'BL_Lymphocyte number (1000 cells/uL)',
    'LBDMONO' : 'BL_Monocyte number (1000 cells/uL)',
    'LBDNENO' : 'BL_Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO' : 'BL_Eosinophils number (1000 cells/uL)',
    'LBDBANO' : 'BL_Basophils number (1000 cells/uL)',
    'LBXRBCSI' : 'BL_Red blood cell count (million cells/uL)',
    'LBXHGB' : 'BL_Hemoglobin (g/dL)',
    'LBXHCT' : 'BL_Hematocrit (%)',
    'LBXMCVSI' : 'BL_Mean cell volume (fL)',
   # 'LBXMCHSI' : 'BL_Mean cell hemoglobin (pg)',
   # 'LBXMC' : 'BL_MCHC (g/dL)',
    'LBXRDW' : 'BL_Red cell distribution width (%)',
    'LBXPLTSI' : 'BL_Platelet count (1000 cells/uL)',
    'LBXMPSI' : 'BL_Mean platelet volume (fL)'
    }
    df_blood = df_blood.rename(columns = col_dict)
    # merge df_blood to df
    df = pd.merge(df, df_blood, how='inner', on=['SEQN'])

    df_bio = pd.read_sas('./NHANES 2013-2014/other NHANES/BIOPRO_'+i+'.XPT')
    col_dict = {
    'SEQN' : 'SEQN',
    'LBDSALSI' : 'BIO_Albumin (g/L)',
    #'LBXSAPSI' : 'BIO_Alkaline phosphatase (IU/L)',
    'LBXSASSI' : 'BIO_Aspartate aminotransferase AST (IU/L)',
    'LBXSATSI' : 'BIO_Alanine aminotransferase ALT (IU/L)',
    'LBXSBU' : 'BIO_Blood urea nitrogen (mg/dL)',
    'LBDSBUSI' : 'BIO_Blood urea nitrogen (mmol/L)',
    'LBXSC3SI' : 'BIO_Bicarbonate (mmol/L)',
    'LBDSCASI' : 'BIO_Total calcium (mmol/L)',
    'LBDSCHSI' : 'BIO_Cholesterol (mmol/L)',
    #'LBXSCK' : 'BIO_Creatine Phosphokinase(CPK) (IU/L)',
    'LBXSCLSI' : 'BIO_Chloride (mmol/L)',
    'LBDSCRSI' : 'BIO_Creatinine (umol/L)',
    'LBDSGBSI' : 'BIO_Globulin (g/L)',
    'LBDSGLSI' : 'BIO_Glucose, refrigerated serum (mmol/L)',
    'LBXSGTSI' : 'BIO_Gamma glutamyl transferase (U/L)',
    'LBDSIRSI' : 'BIO_Iron, refrigerated serum (umol/L)',
    'LBXSKSI' : 'BIO_Potassium (mmol/L)',
   # 'LBXSLDSI' : 'BIO_Lactate dehydrogenase (U/L)',
    'LBXSNASI' : 'BIO_Sodium (mmol/L)',
    'LBXSOSSI' : 'BIO_Osmolality (mmol/Kg)',
    'LBDSPHSI' : 'BIO_Phosphorus (mmol/L)',
    'LBDSTBSI' : 'BIO_Total bilirubin (umol/L)',
    'LBDSTPSI' : 'BIO_Total protein (g/L)',
    'LBDSTRSI' : 'BIO_Triglycerides, refrigerated (mmol/L)',
    'LBDSUASI' : 'BIO_Uric acid (umol/L)'}
    # select the columns i want to keep
    df_bio = df_bio[list(col_dict.keys())].rename(columns=col_dict)
    # merge df_bio with df
    df = pd.merge(df, df_bio, how='inner', on=['SEQN'])
    
    df_final = df_final.append(df)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
df_final['Breast Cancer'].value_counts()

1    656
Name: Breast Cancer, dtype: int64

# do the same for the other women

In [5]:
for i in ['A']:    
    df_demo= pd.read_sas('./NHANES 2013-2014/other NHANES/DEMO_'+i+'.XPT')
    # selecting specific columns
    # selecting gender: women SEQN and Age
    df_demo = df_demo[(df_demo['RIAGENDR'] == 2 ) & (df_demo['RIDAGEYR'] > 10) ][['SEQN', 'RIDAGEYR']]
    df_demo = df_demo.rename(columns={'RIDAGEYR': 'Age'})

    df_med = pd.read_sas('./NHANES 2013-2014/other NHANES/MCQ_'+i+'.XPT')
    # match only dose SEQN matching the one inf df_demo (seqn variable)
    df_med = df_med.loc[df_med['SEQN'].isin(df_demo['SEQN'])]
    df_med = df_med[['SEQN', 'MCQ230A', 'MCQ240E', 'MCQ220']]
    # add column breast cancer 'Diagnosis' 1 no 0
    df_med['Breast Cancer'] = np.where(df_med['MCQ230A'] == 14,1,0)

    # subset to have women that were diagnosed with breast cancer 
    df_med = df_med[df_med['Breast Cancer'] == 0]
    # changing the column name to diagnosis age and saving the dataset
    df_med = df_med.rename(columns={'MCQ240E' : 'Diagnosis Age'})[['SEQN', 'Diagnosis Age', 'Breast Cancer']]
    # merge df_med with df_demo
    df = pd.merge(df_med, df_demo, on='SEQN', how='inner')


    df_blood = pd.read_sas('./NHANES 2013-2014/other NHANES/CBC_'+i+'.XPT')
    # subset to SEQN from df
    df_blood = df_blood.loc[df_blood['SEQN'].isin(df['SEQN'])]
    col_dict = {
    'LBXWBCSI' : 'BL_White blood cell count (1000 cells/uL)',
    'LBXLYPCT' : 'BL_Lymphocyte percent (%)',
    'LBXMOPCT' : 'BL_Monocyte percent (%)',
    'LBXNEPCT' : 'BL_Segmented neutrophils percent (%)',
    'LBXEOPCT' : 'BL_Eosinophils percent (%)',
    'LBXBAPCT' : 'BL_Basophils percent (%)',
    'LBDLYMNO' : 'BL_Lymphocyte number (1000 cells/uL)',
    'LBDMONO' : 'BL_Monocyte number (1000 cells/uL)',
    'LBDNENO' : 'BL_Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO' : 'BL_Eosinophils number (1000 cells/uL)',
    'LBDBANO' : 'BL_Basophils number (1000 cells/uL)',
    'LBXRBCSI' : 'BL_Red blood cell count (million cells/uL)',
    'LBXHGB' : 'BL_Hemoglobin (g/dL)',
    'LBXHCT' : 'BL_Hematocrit (%)',
    'LBXMCVSI' : 'BL_Mean cell volume (fL)',
 #   'LBXMCHSI' : 'BL_Mean cell hemoglobin (pg)',
 #   'LBXMC' : 'BL_MCHC (g/dL)',
    'LBXRDW' : 'BL_Red cell distribution width (%)',
    'LBXPLTSI' : 'BL_Platelet count (1000 cells/uL)',
    'LBXMPSI' : 'BL_Mean platelet volume (fL)'
    }
    df_blood = df_blood.rename(columns = col_dict)
    # merge df_blood to df
    df = pd.merge(df, df_blood, how='inner', on=['SEQN'])

    df_bio = pd.read_sas('./NHANES 2013-2014/other NHANES/BIOPRO_'+i+'.XPT')
    col_dict = {
    'SEQN' : 'SEQN',
    'LBDSALSI' : 'BIO_Albumin (g/L)',
    #'LBXSAPSI' : 'BIO_Alkaline phosphatase (IU/L)',
    'LBXSASSI' : 'BIO_Aspartate aminotransferase AST (IU/L)',
    'LBXSATSI' : 'BIO_Alanine aminotransferase ALT (IU/L)',
    'LBXSBU' : 'BIO_Blood urea nitrogen (mg/dL)',
    'LBDSBUSI' : 'BIO_Blood urea nitrogen (mmol/L)',
    'LBXSC3SI' : 'BIO_Bicarbonate (mmol/L)',
    'LBDSCASI' : 'BIO_Total calcium (mmol/L)',
    'LBDSCHSI' : 'BIO_Cholesterol (mmol/L)',
    #'LBXSCK' : 'BIO_Creatine Phosphokinase(CPK) (IU/L)',
    'LBXSCLSI' : 'BIO_Chloride (mmol/L)',
    'LBDSCRSI' : 'BIO_Creatinine (umol/L)',
    'LBDSGBSI' : 'BIO_Globulin (g/L)',
    'LBDSGLSI' : 'BIO_Glucose, refrigerated serum (mmol/L)',
    'LBXSGTSI' : 'BIO_Gamma glutamyl transferase (U/L)',
    'LBDSIRSI' : 'BIO_Iron, refrigerated serum (umol/L)',
    'LBXSKSI' : 'BIO_Potassium (mmol/L)',
   # 'LBXSLDSI' : 'BIO_Lactate dehydrogenase (U/L)',
    'LBXSNASI' : 'BIO_Sodium (mmol/L)',
    'LBXSOSSI' : 'BIO_Osmolality (mmol/Kg)',
    'LBDSPHSI' : 'BIO_Phosphorus (mmol/L)',
    'LBDSTBSI' : 'BIO_Total bilirubin (umol/L)',
    'LBDSTPSI' : 'BIO_Total protein (g/L)',
    'LBDSTRSI' : 'BIO_Triglycerides, refrigerated (mmol/L)',
    'LBDSUASI' : 'BIO_Uric acid (umol/L)'}
    # select the columns i want to keep
    df_bio = df_bio[list(col_dict.keys())].rename(columns=col_dict)
    # merge df_bio with df
    df = pd.merge(df, df_bio, how='inner', on=['SEQN'])
    
    df_final_h = df.copy()

In [6]:
for i in ['B','C','D','E','F','G','H','I']:    
    df_demo= pd.read_sas('./NHANES 2013-2014/other NHANES/DEMO_'+i+'.XPT')

    # selecting specific columns
    # selecting gender: women SEQN and Age
    df_demo = df_demo[(df_demo['RIAGENDR'] == 2 ) & (df_demo['RIDAGEYR'] > 10) ][['SEQN', 'RIDAGEYR']]
    df_demo = df_demo.rename(columns={'RIDAGEYR': 'Age'})

    df_med = pd.read_sas('./NHANES 2013-2014/other NHANES/MCQ_'+i+'.XPT')
    # match only dose SEQN matching the one inf df_demo (seqn variable)
    df_med = df_med.loc[df_med['SEQN'].isin(df_demo['SEQN'])]
    df_med = df_med[['SEQN', 'MCQ230A', 'MCQ240E', 'MCQ220']]
    # add column breast cancer 'Diagnosis' 1 no 0
    df_med['Breast Cancer'] = np.where(df_med['MCQ230A'] == 14,1,0)

    # subset to have women that were diagnosed with breast cancer 
    df_med = df_med[df_med['Breast Cancer'] == 0]
    # changing the column name to diagnosis age and saving the dataset
    df_med = df_med.rename(columns={'MCQ240E' : 'Diagnosis Age'})[['SEQN', 'Diagnosis Age', 'Breast Cancer']]
    # merge df_med with df_demo
    df = pd.merge(df_med, df_demo, on='SEQN', how='inner')
    
    
    df_blood = pd.read_sas('./NHANES 2013-2014/other NHANES/CBC_'+i+'.XPT')
    # subset to SEQN from df
    df_blood = df_blood.loc[df_blood['SEQN'].isin(df['SEQN'])]
    col_dict = {
    'LBXWBCSI' : 'BL_White blood cell count (1000 cells/uL)',
    'LBXLYPCT' : 'BL_Lymphocyte percent (%)',
    'LBXMOPCT' : 'BL_Monocyte percent (%)',
    'LBXNEPCT' : 'BL_Segmented neutrophils percent (%)',
    'LBXEOPCT' : 'BL_Eosinophils percent (%)',
    'LBXBAPCT' : 'BL_Basophils percent (%)',
    'LBDLYMNO' : 'BL_Lymphocyte number (1000 cells/uL)',
    'LBDMONO' : 'BL_Monocyte number (1000 cells/uL)',
    'LBDNENO' : 'BL_Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO' : 'BL_Eosinophils number (1000 cells/uL)',
    'LBDBANO' : 'BL_Basophils number (1000 cells/uL)',
    'LBXRBCSI' : 'BL_Red blood cell count (million cells/uL)',
    'LBXHGB' : 'BL_Hemoglobin (g/dL)',
    'LBXHCT' : 'BL_Hematocrit (%)',
    'LBXMCVSI' : 'BL_Mean cell volume (fL)',
 #   'LBXMCHSI' : 'BL_Mean cell hemoglobin (pg)',
 #   'LBXMC' : 'BL_MCHC (g/dL)',
    'LBXRDW' : 'BL_Red cell distribution width (%)',
    'LBXPLTSI' : 'BL_Platelet count (1000 cells/uL)',
    'LBXMPSI' : 'BL_Mean platelet volume (fL)'
    }
    df_blood = df_blood.rename(columns = col_dict)
    # merge df_blood to df
    df = pd.merge(df, df_blood, how='inner', on=['SEQN'])

    df_bio = pd.read_sas('./NHANES 2013-2014/other NHANES/BIOPRO_'+i+'.XPT')
    col_dict = {
    'SEQN' : 'SEQN',
    'LBDSALSI' : 'BIO_Albumin (g/L)',
    #'LBXSAPSI' : 'BIO_Alkaline phosphatase (IU/L)',
    'LBXSASSI' : 'BIO_Aspartate aminotransferase AST (IU/L)',
    'LBXSATSI' : 'BIO_Alanine aminotransferase ALT (IU/L)',
    'LBXSBU' : 'BIO_Blood urea nitrogen (mg/dL)',
    'LBDSBUSI' : 'BIO_Blood urea nitrogen (mmol/L)',
    'LBXSC3SI' : 'BIO_Bicarbonate (mmol/L)',
    'LBDSCASI' : 'BIO_Total calcium (mmol/L)',
    'LBDSCHSI' : 'BIO_Cholesterol (mmol/L)',
    #'LBXSCK' : 'BIO_Creatine Phosphokinase(CPK) (IU/L)',
    'LBXSCLSI' : 'BIO_Chloride (mmol/L)',
    'LBDSCRSI' : 'BIO_Creatinine (umol/L)',
    'LBDSGBSI' : 'BIO_Globulin (g/L)',
    'LBDSGLSI' : 'BIO_Glucose, refrigerated serum (mmol/L)',
    'LBXSGTSI' : 'BIO_Gamma glutamyl transferase (U/L)',
    'LBDSIRSI' : 'BIO_Iron, refrigerated serum (umol/L)',
    'LBXSKSI' : 'BIO_Potassium (mmol/L)',
   # 'LBXSLDSI' : 'BIO_Lactate dehydrogenase (U/L)',
    'LBXSNASI' : 'BIO_Sodium (mmol/L)',
    'LBXSOSSI' : 'BIO_Osmolality (mmol/Kg)',
    'LBDSPHSI' : 'BIO_Phosphorus (mmol/L)',
    'LBDSTBSI' : 'BIO_Total bilirubin (umol/L)',
    'LBDSTPSI' : 'BIO_Total protein (g/L)',
    'LBDSTRSI' : 'BIO_Triglycerides, refrigerated (mmol/L)',
    'LBDSUASI' : 'BIO_Uric acid (umol/L)'}
    # select the columns i want to keep
    df_bio = df_bio[list(col_dict.keys())].rename(columns=col_dict)
    # merge df_bio with df
    df = pd.merge(df, df_bio, how='inner', on=['SEQN'])
    
    df_final_h = df_final_h.append(df)

In [7]:
df_final_h['Breast Cancer'].value_counts()

0    31503
Name: Breast Cancer, dtype: int64

In [8]:
df_final_final = df_final.append(df_final_h)

In [9]:
df_final_final['Breast Cancer'].value_counts()

0    31503
1      656
Name: Breast Cancer, dtype: int64

# Clean up

# create 2 subsets from women that were diagnosed. one for biochem one for blood 

In [24]:
df = df_final_final.copy()
# clean column names
df.columns = df.columns.str.lower().str.capitalize()
#rename the Seqn column
df = df.rename(columns={'Seqn':'Id'})

# BIOCHEM

In [25]:
# column names for subset BIO
df_bio_col = ['Id', 'Age']
for i in list(df.columns):
    if i.startswith('Bio'):
        df_bio_col.append(i)
df_bio_col.append('Breast cancer')

df_bio = df[df_bio_col]

In [26]:
df_bio_bcancer = df_bio[df_bio['Breast cancer'] == 1]
df_bio_nbcancer = df_bio[df_bio['Breast cancer'] == 0]

In [27]:
df_bio_bcancer.isna().sum()

Id                                            0
Age                                           0
Bio_alanine aminotransferase alt (iu/l)      90
Bio_albumin (g/l)                            90
Bio_aspartate aminotransferase ast (iu/l)    90
Bio_bicarbonate (mmol/l)                     90
Bio_blood urea nitrogen (mg/dl)              90
Bio_blood urea nitrogen (mmol/l)             90
Bio_chloride (mmol/l)                        90
Bio_cholesterol (mmol/l)                     90
Bio_creatinine (umol/l)                      90
Bio_gamma glutamyl transferase (u/l)         90
Bio_globulin (g/l)                           90
Bio_glucose, refrigerated serum (mmol/l)     90
Bio_iron, refrigerated serum (umol/l)        90
Bio_osmolality (mmol/kg)                     90
Bio_phosphorus (mmol/l)                      90
Bio_potassium (mmol/l)                       90
Bio_sodium (mmol/l)                          90
Bio_total bilirubin (umol/l)                 90
Bio_total calcium (mmol/l)              

In [28]:
# there are 90 values missing from many columns, lets just drop them
df_bio_bcancer = df_bio_bcancer.dropna(thresh=20)

In [29]:
df_bio_bcancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566 entries, 1 to 81
Data columns (total 25 columns):
Id                                           566 non-null float64
Age                                          566 non-null float64
Bio_alanine aminotransferase alt (iu/l)      566 non-null float64
Bio_albumin (g/l)                            566 non-null float64
Bio_aspartate aminotransferase ast (iu/l)    566 non-null float64
Bio_bicarbonate (mmol/l)                     566 non-null float64
Bio_blood urea nitrogen (mg/dl)              566 non-null float64
Bio_blood urea nitrogen (mmol/l)             566 non-null float64
Bio_chloride (mmol/l)                        566 non-null float64
Bio_cholesterol (mmol/l)                     566 non-null float64
Bio_creatinine (umol/l)                      566 non-null float64
Bio_gamma glutamyl transferase (u/l)         566 non-null float64
Bio_globulin (g/l)                           566 non-null float64
Bio_glucose, refrigerated serum (mmol/

In [30]:
# remove the NaNs from the set without breast cancer
df_bio_nbcancer = df_bio_nbcancer.dropna(thresh=20)
# there are 566 entries, subset random 566 from the women that were not diangosed
df_bio_nbcancer = df_bio_nbcancer.sample(n=566)

# same for blood

In [31]:
# column names for subset BL ( blood )
df_bl_col = ['Id', 'Age']
for i in list(df.columns):
    if i.startswith('Bl'):
        df_bl_col.append(i)
df_bl_col.append('Breast cancer')

df_bl = df[df_bl_col]

In [32]:
df_bl_bcancer = df_bl[df_bl['Breast cancer'] == 1]
df_bl_nbcancer = df_bl[df_bl['Breast cancer'] == 0]

In [33]:
df_bl_bcancer.isna().sum()

Id                                              0
Age                                             0
Bl_basophils number (1000 cells/ul)            73
Bl_basophils percent (%)                       73
Bl_eosinophils number (1000 cells/ul)          73
Bl_eosinophils percent (%)                     73
Bl_hematocrit (%)                              69
Bl_hemoglobin (g/dl)                           69
Bl_lymphocyte number (1000 cells/ul)           73
Bl_lymphocyte percent (%)                      73
Bl_mean cell volume (fl)                       69
Bl_mean platelet volume (fl)                   69
Bl_monocyte number (1000 cells/ul)             73
Bl_monocyte percent (%)                        73
Bl_platelet count (1000 cells/ul)              69
Bl_red blood cell count (million cells/ul)     69
Bl_red cell distribution width (%)             69
Bl_segmented neutrophils num (1000 cell/ul)    73
Bl_segmented neutrophils percent (%)           73
Bl_white blood cell count (1000 cells/ul)      69


In [34]:
# many missing values
df_bl_bcancer = df_bl_bcancer.dropna(thresh=20)


In [35]:
df_bl_bcancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 1 to 81
Data columns (total 21 columns):
Id                                             583 non-null float64
Age                                            583 non-null float64
Bl_basophils number (1000 cells/ul)            583 non-null float64
Bl_basophils percent (%)                       583 non-null float64
Bl_eosinophils number (1000 cells/ul)          583 non-null float64
Bl_eosinophils percent (%)                     583 non-null float64
Bl_hematocrit (%)                              583 non-null float64
Bl_hemoglobin (g/dl)                           583 non-null float64
Bl_lymphocyte number (1000 cells/ul)           583 non-null float64
Bl_lymphocyte percent (%)                      583 non-null float64
Bl_mean cell volume (fl)                       583 non-null float64
Bl_mean platelet volume (fl)                   583 non-null float64
Bl_monocyte number (1000 cells/ul)             583 non-null float64
Bl_monocyte 

In [36]:
# remove the NaNs from the set without breast cancer
df_bl_nbcancer = df_bl_nbcancer.dropna(thresh=20)

# there are 583 entries, subset random 583 from the women that were not diagnosed
df_bl_nbcancer = df_bl_nbcancer.sample(n=583)

In [37]:
df_bl_nbcancer.isna().sum()

Id                                             0
Age                                            0
Bl_basophils number (1000 cells/ul)            0
Bl_basophils percent (%)                       0
Bl_eosinophils number (1000 cells/ul)          0
Bl_eosinophils percent (%)                     0
Bl_hematocrit (%)                              0
Bl_hemoglobin (g/dl)                           0
Bl_lymphocyte number (1000 cells/ul)           0
Bl_lymphocyte percent (%)                      0
Bl_mean cell volume (fl)                       0
Bl_mean platelet volume (fl)                   0
Bl_monocyte number (1000 cells/ul)             0
Bl_monocyte percent (%)                        0
Bl_platelet count (1000 cells/ul)              0
Bl_red blood cell count (million cells/ul)     0
Bl_red cell distribution width (%)             0
Bl_segmented neutrophils num (1000 cell/ul)    0
Bl_segmented neutrophils percent (%)           0
Bl_white blood cell count (1000 cells/ul)      0
Breast cancer       

In [41]:
# check for duplicates
df_bl.duplicated().sum() , df_bio.duplicated().sum()

(0, 0)

In [ ]:
# create final datasets
df_bl = df_bl_nbcancer.append(df_bl_bcancer).sort_values(by=['Id'])
df_bl.to_csv('nhanes_breast_cancer_all_nhanes_blood.csv', index=False)
df_bio = df_bio_nbcancer.append(df_bio_bcancer).sort_values(by=['Id'])
df_bio.to_csv('nhanes_breast_cancer_all_nhanes_biochemistry.csv', index=False)